In [1]:
import pandas as pd
import funzioni as fx
from multiprocessing import Pool, cpu_count, Manager, Value
import numpy as np
import os

def process_ticker(nome_simbolo, features_prezzo, features_da_scalare_singolarmente, features_meno_piu, features_candele, features_no_scala, elenco_targets, n_timesteps, giorni_previsione, bilanciamento):
    try:
        ticker = pd.read_parquet(f"tickers/{nome_simbolo}.parquet")
        _, X, Y, _ = fx.to_XY(ticker, features_prezzo, features_da_scalare_singolarmente, features_meno_piu, features_candele, features_no_scala, elenco_targets, n_timesteps, giorni_previsione, bilanciamento)
        return nome_simbolo, X, Y, ""
    except Exception as e:
        return nome_simbolo, None, None, str(e)

def callback_result(result, listaX, listaY, perc_dati, set_file_x_y, totale_processati, n_simboli_addestramento):
    nome_simbolo, X, Y, err = result
    if X is not None and Y is not None:
        print(f'X.shape:{X.shape}')
        print(f'Y.shape:{Y.shape}')
        listaX.append(X)
        listaY.append(Y)
        print("\033[42m" + f"{totale_processati.value}/{n_simboli_addestramento}) Completato ticker {nome_simbolo}" + "\033[0m")
    else:
        print(err)

    with totale_processati.get_lock(): 
        totale_processati.value += 1
        if (totale_processati.value >= n_simboli_addestramento):
            print("Concatenamento Y")
            Ytot = np.concatenate(listaY, axis=0)
            print("Concatenamento X")
            Xtot = np.concatenate(listaX, axis=0)
            print(f"X.shape:{Xtot.shape}")
            print(f"Y.shape:{Ytot.shape}")
            print("Salvataggio su file")
            np.save(f'{perc_dati}/X{set_file_x_y}', Xtot)
            np.save(f'{perc_dati}/Y{set_file_x_y}', Ytot)

def prepara_dati(lista_ticker, n_simboli_addestramento, perc_dati, set_file_x_y, bilanciamento=0):
    manager = Manager()
    listaX = manager.list()
    listaY = manager.list()
    totale_processati = Value('i', 0)  
   
    with Pool(cpu_count()) as p:
        for i in range(totale_processati.value, n_simboli_addestramento):
            nome_simbolo = lista_ticker.iloc[i]["Ticker"]
            task = (nome_simbolo, fx.features_prezzo, fx.features_da_scalare_singolarmente, fx.features_meno_piu, fx.features_candele, fx.features_no_scala, fx.elenco_targets, fx.n_timesteps, fx.giorni_previsione, bilanciamento)
            p.apply_async(process_ticker, args=task, callback=lambda result: callback_result(result, listaX, listaY, perc_dati, set_file_x_y, totale_processati, n_simboli_addestramento))

        p.close()
        p.join()

lista_ticker = pd.read_parquet("lista_ticker.parquet")
lista_ticker.sample(frac=1)
tot_ticker = 500
bilanciamento = 1
prepara_dati(lista_ticker, tot_ticker, 'dati', f'_{tot_ticker}_{bilanciamento}', bilanciamento=bilanciamento)


Using TensorFlow backend
X.shape:(484, 120, 10)
Y.shape:(484, 1)
0/500) Completato ticker CARR
X.shape:(78, 120, 10)
Y.shape:(78, 1)
1/500) Completato ticker ZIP
X.shape:(1104, 120, 10)
Y.shape:(1104, 1)
2/500) Completato ticker OOMA
X.shape:(1044, 120, 10)
Y.shape:(1044, 1)
3/500) Completato ticker IRTC
X.shape:(908, 120, 10)
Y.shape:(908, 1)
4/500) Completato ticker NGVT
X.shape:(748, 120, 10)
Y.shape:(748, 1)
5/500) Completato ticker SCPL
X.shape:(2410, 120, 10)
Y.shape:(2410, 1)
6/500) Completato ticker PPBI
X.shape:(246, 120, 10)
Y.shape:(246, 1)
7/500) Completato ticker DV
X.shape:(2524, 120, 10)
Y.shape:(2524, 1)
8/500) Completato ticker CE
X.shape:(1646, 120, 10)
Y.shape:(1646, 1)
9/500) Completato ticker YPF
X.shape:(1710, 120, 10)
Y.shape:(1710, 1)
10/500) Completato ticker CHEF
X.shape:(214, 120, 10)
Y.shape:(214, 1)
11/500) Completato ticker NABL
X.shape:(2118, 120, 10)
Y.shape:(2118, 1)
12/500) Completato ticker PII
X.shape:(2576, 120, 10)
Y.shape:(2576, 1)
13/500) Complet